In [1]:
import scipy.signal as signal
from sklearn.model_selection import KFold, train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, LambdaCallback
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import pandas as pd

import wandb
wandb.login()

import os
import numpy as np
from scipy import signal

from tensorflow.keras import layers, models, regularizers

def create_model(config):
    kernel_size = config.kernel_size
    regularizer = regularizers.l2(config.reg)
    dropout_rate = config.dropout
    size_0 = config.start_filters

    # Input layer
    in_data = layers.Input(shape=(1024, 1))

    # Encoder part
    conv0 = layers.Conv1D(size_0, kernel_size, kernel_regularizer=regularizer, activation='relu', padding='same', kernel_initializer='he_normal')(in_data)
    conv0 = layers.BatchNormalization()(conv0)
    conv0 = layers.Conv1D(size_0, kernel_size, kernel_regularizer=regularizer, activation='relu', padding='same', kernel_initializer='he_normal')(conv0)
    conv0 = layers.BatchNormalization()(conv0)
    pool0 = layers.MaxPooling1D(pool_size=2)(conv0)

    size_1 = size_0 * 2
    conv1 = layers.Conv1D(size_1, kernel_size, kernel_regularizer=regularizer, activation='relu', padding='same', kernel_initializer='he_normal')(pool0)
    conv1 = layers.BatchNormalization()(conv1)
    conv1 = layers.Conv1D(size_1, kernel_size, kernel_regularizer=regularizer, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    conv1 = layers.BatchNormalization()(conv1)
    conv1 = layers.Dropout(dropout_rate)(conv1)
    pool1 = layers.MaxPooling1D(pool_size=2)(conv1)

    size_2 = size_1 * 2
    conv2 = layers.Conv1D(size_2, kernel_size, kernel_regularizer=regularizer, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = layers.BatchNormalization()(conv2)
    conv2 = layers.Conv1D(size_2, kernel_size, kernel_regularizer=regularizer, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    conv2 = layers.BatchNormalization()(conv2)

    # Decoder part
    up1 = layers.UpSampling1D(size=2)(conv2)
    up_conv1 = layers.Conv1D(size_2, kernel_size, activation='relu', padding='same', kernel_initializer='he_normal')(up1)
    up_conv1 = layers.BatchNormalization()(up_conv1)
    merge1 = layers.concatenate([conv1, up_conv1], axis=2)
    conv3 = layers.Conv1D(size_1, kernel_size, kernel_regularizer=regularizer, activation='relu', padding='same', kernel_initializer='he_normal')(merge1)
    conv3 = layers.BatchNormalization()(conv3)
    conv3 = layers.Conv1D(size_1, kernel_size, kernel_regularizer=regularizer, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    conv3 = layers.BatchNormalization()(conv3)
    conv3 = layers.Dropout(dropout_rate)(conv3)

    up2 = layers.UpSampling1D(size=2)(conv3)
    up_conv2 = layers.Conv1D(size_1, kernel_size, activation='relu', padding='same', kernel_initializer='he_normal')(up2)
    up_conv2 = layers.BatchNormalization()(up_conv2)
    merge2 = layers.concatenate([conv0, up_conv2], axis=2)
    conv4 = layers.Conv1D(size_0, kernel_size, kernel_regularizer=regularizer, activation='relu', padding='same', kernel_initializer='he_normal')(merge2)
    conv4 = layers.BatchNormalization()(conv4)
    conv4 = layers.Conv1D(size_0, kernel_size, kernel_regularizer=regularizer, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
    conv4 = layers.BatchNormalization()(conv4)

    # Output layer
    out_data = layers.Conv1D(1, kernel_size, activation='sigmoid', padding='same')(conv4)

    model = models.Model(inputs=[in_data], outputs=[out_data])
    model.summary()

    return model

def create_sweep_config():
    return {
        'method': 'bayes',  # or 'random', 'grid'
        'metric': {
            'name': 'val_loss',
            'goal': 'minimize'
        },
        'parameters': {
            'learning_rate': {
                'min': 1e-5,
                'max': 1e-2
            },
            'batch_size': {
                'values': [32, 64, 128, 256]
            },
            'kernel_size': {
                'values': [3, 5, 7, 27]
            },
            'reg': {
                'min': 1e-5,
                'max': 1e-2
            },
            'dropout': {
                'min': 0.1,
                'max': 0.6
            },
            'start_filters': {
                'values': [8, 16, 32]
            }
        }
    }


def sliding_window(data, window_size, downsampled_window_size, overlap, train_patients, validation_patients, test_patients):
    windows_ecg_train = []
    windows_resp_train = []

    for train_patient in train_patients:
    
        N = len(data[train_patient][0])
        max_step = int(N//(window_size*overlap))
        for step in range(max_step):
            ecg = data[train_patient][0][step * int(window_size*overlap):step * int(window_size*overlap) + window_size] 
            resp = data[train_patient][1][step * int(window_size*overlap):step * int(window_size*overlap) + window_size]
            
            if (ecg.min() < ecg.max()):
                normalized_ecg = (ecg-ecg.min())/(ecg.max()-ecg.min())-0.5
                #zero_centered_ecg = ecg - np.mean(ecg)
                #normalized_ecg = zero_centered_ecg / np.std(zero_centered_ecg)
                resampled_ecg = signal.resample(normalized_ecg, downsampled_window_size)
                if resp.min() < resp.max():
                    normalized_resp = (resp-resp.min())/(resp.max()-resp.min())
                    #zero_centered_resp = resp - np.mean(resp)
                    #normalized_resp = zero_centered_resp / np.std(zero_centered_resp)
                    resampled_resp = signal.resample(normalized_resp, downsampled_window_size)
                    windows_ecg_train.append(np.float32(resampled_ecg))
                    windows_resp_train.append(np.float32(resampled_resp))
            
            
    windows_ecg_validation = []
    windows_resp_validation = []


    for validation_patient in validation_patients:
        N = len(data[validation_patient][0])
        max_step = int(N//(window_size*overlap))
        for step in range(max_step):
            ecg = data[validation_patient][0][step * int(window_size*overlap):step * int(window_size*overlap) + window_size] 
            resp = data[validation_patient][1][step * int(window_size*overlap):step * int(window_size*overlap) + window_size]
            
            if (ecg.min() < ecg.max()):
                normalized_ecg = (ecg-ecg.min())/(ecg.max()-ecg.min())-0.5
                #zero_centered_ecg = ecg - np.mean(ecg)
                #normalized_ecg = zero_centered_ecg / np.std(zero_centered_ecg)
                resampled_ecg = signal.resample(normalized_ecg, downsampled_window_size)
                if resp.min() < resp.max():
                    normalized_resp = (resp-resp.min())/(resp.max()-resp.min())
                    #zero_centered_resp = resp - np.mean(resp)
                    #normalized_resp = zero_centered_resp / np.std(zero_centered_resp)
                    resampled_resp = signal.resample(normalized_resp, downsampled_window_size)
                    windows_ecg_validation.append(np.float32(resampled_ecg))
                    windows_resp_validation.append(np.float32(resampled_resp))
          
    windows_ecg_test = []
    windows_resp_test = []
    
    for test_patient in test_patients:
        N = len(data[test_patient][0])
        max_step = int(N//(window_size*overlap))
        for step in range(max_step):
            ecg = data[test_patient][0][step * int(window_size*overlap):step * int(window_size*overlap) + window_size] 
            resp = data[test_patient][1][step * int(window_size*overlap):step * int(window_size*overlap) + window_size]
            
            if (ecg.min() < ecg.max()):
                normalized_ecg = (ecg-ecg.min())/(ecg.max()-ecg.min())-0.5
                #zero_centered_ecg = ecg - np.mean(ecg)
                #normalized_ecg = zero_centered_ecg / np.std(zero_centered_ecg)
                resampled_ecg = signal.resample(normalized_ecg, downsampled_window_size)
                if resp.min() < resp.max():
                    normalized_resp = (resp-resp.min())/(resp.max()-resp.min())
                    #zero_centered_resp = resp - np.mean(resp)
                    #normalized_resp = zero_centered_resp / np.std(zero_centered_resp)
                    resampled_resp = signal.resample(normalized_resp, downsampled_window_size)
                    windows_ecg_test.append(np.float32(resampled_ecg))
                    windows_resp_test.append(np.float32(resampled_resp))

    windows_ecg_train = np.stack(windows_ecg_train, axis=0)
    windows_resp_train = np.stack(windows_resp_train, axis=0)
    windows_ecg_validation = np.stack(windows_ecg_validation, axis=0)
    windows_resp_validation = np.stack(windows_resp_validation, axis=0)
    windows_ecg_test = np.stack(windows_ecg_test, axis=0)
    windows_resp_test = np.stack(windows_resp_test, axis=0)

    windows_ecg_train = windows_ecg_train[:,:,np.newaxis]
    windows_resp_train = windows_resp_train[:,:,np.newaxis]
    windows_ecg_validation = windows_ecg_validation[:,:,np.newaxis]
    windows_resp_validation = windows_resp_validation[:,:,np.newaxis]
    windows_ecg_test = windows_ecg_test[:,:,np.newaxis]
    windows_resp_test = windows_resp_test[:,:,np.newaxis]

    return windows_ecg_train, windows_resp_train, windows_ecg_validation, windows_resp_validation, windows_ecg_test, windows_resp_test

def load_data():
     # bidmc
    path = "/Users/lanacaldarevic/workspace/phd/ecg_derived_resp_dl/data/bidmc-ppg-and-respiration-dataset-1.0.0"
    EXT = "*Signals.csv"
    all_csv_files = [file for path, subdir, files in os.walk(path) for file in glob(os.path.join(path, EXT))]
    patients = []
    data = {}
    no_errors = 0
    for file in all_csv_files:
        try:
            df = pd.read_csv(file)
            X1, X2, X3, X4 = df[' PLETH'].values, df[' V'].values, df[' AVR'].values, df[' II'].values
            # X = np.concatenate([X1.reshape(len(X1),1),X2.reshape(len(X1),1),X3.reshape(len(X1),1),X4.reshape(len(X1),1)], axis=1)
            
            Y = df[' RESP'].values
            
            patient = int(file.split('/')[-1].split('_')[1])
            patients.append(patient)
            data[patient] = [X4, Y]
        except:
            no_errors += 1

    return data, patients


def visualize_predictions(set_name, model, windows_ecg, windows_resp):
    prediction = model.predict(windows_ecg)[0]
    windows_resp= tf.squeeze(windows_resp)

    fig, ax = plt.subplots(2, 1, figsize=(10, 4))
    ax[0].plot(windows_resp)
    ax[0].set_title(f'{set_name.capitalize()} Ground Truth')
    ax[1].plot(prediction)
    ax[1].set_title(f'{set_name.capitalize()} Prediction')

    wandb.log({f"{set_name}_predictions": wandb.Image(fig)}, commit=False)
    plt.close(fig)

def log_images(model, epoch, logs, fixed_sample_ecg_train, fixed_sample_resp_train, fixed_sample_ecg_valid, fixed_sample_resp_valid):
        wandb.log({"loss": logs['loss'], "correlation": logs['correlation'], "val_loss": logs['val_loss'], "val_correlation": logs['val_correlation']})
        #visualize_predictions('train', model, fixed_sample_ecg_train, fixed_sample_resp_train)
        #visualize_predictions('validation', model, fixed_sample_ecg_valid, fixed_sample_resp_valid)

def correlation(x, y): #todo: check this and see in papers what cross correlation is
    # Normalize y to the [0, 1] range
    min_y = tf.math.reduce_min(y)
    max_y = tf.math.reduce_max(y)
    r_up = tf.math.subtract(y, min_y)
    r_down = max_y - min_y
    new_y = r_up / r_down
    
    # Compute means
    mx = tf.math.reduce_mean(x)
    my = tf.math.reduce_mean(new_y)
    
    # Compute centered values
    xm, ym = x - mx, new_y - my
    
    # Compute correlation coefficient
    r_num = tf.reduce_sum(tf.multiply(xm, ym))
    r_den = tf.sqrt(tf.multiply(tf.reduce_sum(tf.square(xm)), tf.reduce_sum(tf.square(ym))))
    r = r_num / r_den
    
    # Ensure the result is between -1 and 1
    r = tf.maximum(tf.minimum(r, 1.0), -1.0)
    
    return 1 - r

def cross_correlation(x, y):
    mean_x = tf.reduce_mean(x)
    mean_y = tf.reduce_mean(y)
    
    centered_x = x - mean_x
    centered_y = y - mean_y
    
    numerator = tf.reduce_sum(centered_x * tf.roll(centered_y, shift=1, axis=0))
    denominator = tf.sqrt(tf.reduce_sum(centered_x**2) * tf.reduce_sum(tf.roll(centered_y, shift=1, axis=0)**2))
    
    return numerator / denominator

def train():
    wandb.init()
    config = wandb.config

    sampling_rate = 125
    input_size_seconds = 16 # //2, *2
    downsampled_window_size = 1024 #? power of 2
    window_size = input_size_seconds * sampling_rate
    overlap = 0.25 #25%
    
    data, patients = load_data()

    # k-folds for CV
    train_patients = []
    test_patients = []
    validation_patients = []
    k = 5
    kf = KFold(n_splits=k)
    train_ind = []
    test_ind = []

    for tr_ind, te_ind in kf.split(patients):
        train_ind.append(tr_ind)
        test_ind.append(te_ind)

    split_ind = 0
    train_index, test_index = train_ind[split_ind], test_ind[split_ind]
    test_index, validation_index = train_test_split(test_index, test_size=0.5, random_state=42)
    train_patients = [patients[i] for i in train_index]
    validation_patients = [patients[i] for i in validation_index]
    test_patients = [patients[i] for i in test_index]

    windows_ecg_train, windows_resp_train, windows_ecg_validation, windows_resp_validation, windows_ecg_test, windows_resp_test = sliding_window(data, window_size, downsampled_window_size, overlap, train_patients, validation_patients, test_patients)

    if tf.config.list_physical_devices('GPU'):
        print("GPU enabled")
        with tf.device('/GPU:0'):
            windows_ecg_train = tf.convert_to_tensor(windows_ecg_train, dtype=tf.float32)
            windows_resp_train = tf.convert_to_tensor(windows_resp_train, dtype=tf.float32)
            windows_ecg_validation = tf.convert_to_tensor(windows_ecg_validation, dtype=tf.float32)
            windows_resp_validation = tf.convert_to_tensor(windows_resp_validation, dtype=tf.float32)
            windows_ecg_test = tf.convert_to_tensor(windows_ecg_test, dtype=tf.float32)
            windows_resp_test = tf.convert_to_tensor(windows_resp_test, dtype=tf.float32)

    fixed_index = 5
    fixed_sample_ecg_train = tf.expand_dims(windows_ecg_train[fixed_index], axis=0)
    fixed_sample_resp_train = tf.expand_dims(windows_resp_train[fixed_index], axis=0)

    fixed_sample_ecg_valid = tf.expand_dims(windows_ecg_validation[fixed_index], axis=0)
    fixed_sample_resp_valid = tf.expand_dims(windows_resp_validation[fixed_index], axis=0)
    
    lr = wandb.config.learning_rate
    optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999, amsgrad=False)
    
    model = create_model(config)
    
    # define callbacks
    #reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, cooldown=5, mode='min', min_lr=1e-7)
    #filepath = os.path.join('Users/lanacaldarevic/ecg_derived_resp_dl/models', f'model_crossval{split_ind}-size{start_filters}-input{input_size}_weights-improvement.h5')
    #checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    
    # early stopping
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, start_from_epoch=20)
    lambda_cb = LambdaCallback(on_epoch_end=lambda epoch, logs: log_images(model, epoch, logs, fixed_sample_ecg_train, fixed_sample_resp_train, fixed_sample_ecg_valid, fixed_sample_resp_valid))

    callbacks = [wandb.keras.WandbCallback(), early_stopping]
    loss = tf.keras.losses.Huber(delta=0.1)
    model.compile(loss=loss, metrics=[correlation, 'mse'], optimizer=optimizer)
    
    print("Model training starting")
    model.fit(windows_ecg_train, windows_resp_train,
              epochs=200,
              batch_size=wandb.config.batch_size,
              shuffle=True,
              callbacks=callbacks,
              validation_data=(windows_ecg_validation, windows_resp_validation))
    
    model.save(os.path.join('Users/lanacaldarevic/ecg_derived_resp_dl/models', f'combined_model{split_ind}-size{config.start_filters}-input{1024}.h5'))

    wandb.finish()

wandb: Currently logged in as: lana-caldarevic1. Use `wandb login --relogin` to force relogin


In [2]:
sweep_config = create_sweep_config()
sweep_id = wandb.sweep(sweep_config, project="biosignal_deeplearning")

# Run sweep
wandb.agent(sweep_id, train)

Create sweep with ID: 6i1t9675
Sweep URL: https://wandb.ai/lana-caldarevic1/biosignal_deeplearning/sweeps/6i1t9675


wandb: Agent Starting Run: pd934tfc with config:
wandb: 	batch_size: 256
wandb: 	dropout: 0.10147334313679082
wandb: 	kernel_size: 27
wandb: 	learning_rate: 0.008925129761412007
wandb: 	reg: 0.00619015941648575
wandb: 	start_filters: 8


GPU enabled


2024-04-10 13:38:32.910348: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-04-10 13:38:32.910421: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-04-10 13:38:32.910432: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-04-10 13:38:32.910706: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-10 13:38:32.910738: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1024, 1)]            0         []                            
                                                                                                  
 conv1d (Conv1D)             (None, 1024, 8)              224       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 1024, 8)              32        ['conv1d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 conv1d_1 (Conv1D)           (None, 1024, 8)              1736      ['batch_normalization[0][0

...

INFO:tensorflow:Assets written to: /Users/lanacaldarevic/workspace/phd/ecg_derived_resp_dl/notebooks/wandb/run-20240410_144158-syvq53cp/files/model-best/assets
wandb: Adding directory to artifact (/Users/lanacaldarevic/workspace/phd/ecg_derived_resp_dl/notebooks/wandb/run-20240410_144158-syvq53cp/files/model-best)... Done. 0.1s


68/68 [==============================] - 70s 1s/step - loss: 0.0604 - correlation: 0.8926 - mse: 0.1162 - val_loss: 0.0628 - val_correlation: 0.9794 - val_mse: 0.1233
Epoch 31/200
48/68 [====================>.........] - ETA: 12:50 - loss: 0.0596 - correlation: 0.8952 - mse: 0.1158

wandb: Ctrl + C detected. Stopping sweep.


Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x313176310>> (for post_run_cell), with arguments args (<ExecutionResult object at 1074f7ca0, execution_count=2 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1074f7ac0, raw_cell="sweep_config = create_sweep_config()
sweep_id = wa.." store_history=True silent=False shell_futures=True cell_id=684f5a87-d8aa-4306-b87c-024af70b337c> result=None>,),kwargs {}:


TypeError: _pause_backend() takes 1 positional argument but 2 were given